# Предисловие

Попробуем сделать по-человечески, все будет расписано (надеюсь) и этот код будет довольно дружелюбен 

P.S. Я не знаю, что из этого получиться и как долго хватит моего интузиазма 


# Библиотеки

In [33]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.optimize import root

# Задание №1 
Напишите две функции:

проводящую PCA при помощи собственных значений,

проводящую PCA при помощи SVD.

добавьте в функции возможность автоматического выбора нужного числа главных компонент. 

Сравните результаты между собой и с аналогичной функцией из scikit-learn.

## PCA через собственные значения

In [34]:
def my_pca_eig(X, n_components=None, variance_threshold=0.95): 
    """
    Выполняет PCA на матрице данных X, в общих словах, где n_components - это количество главных компонент (заранее None - это автоматический выбор)
    а variance_threshold порог кумулятивной объяснённой дисперсии 
    """

    mean = np.mean(X, axis=0) # вычисляем средние по столбцам
    X_centered = X - mean # вычисляем средние из каждого столбца, наши данные центрированы вокруг 0

    n_samples = X.shape[0]
    cov_matrix = X_centered.T @ X_centered / (n_samples - 1) # Ковариационная матрица: (X'X)/(n-1)

    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix) # Разложение на собственные значения и векторы
    
    idx = np.argsort(eigenvalues)[::-1] # Сортировка по убыванию
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # Автоматический выбор числа компонент
    if n_components is None:
        cumsum_var = np.cumsum(eigenvalues) / np.sum(eigenvalues)
        n_components = np.argmax(cumsum_var >= variance_threshold) + 1

    components = eigenvectors[:, :n_components] # Выбираем первые n_components
    transformed = X_centered @ components
    explained_variance = eigenvalues[:n_components]

    return transformed, explained_variance, components, mean


### Примечание 

Без масштабирования, чувствительна к шкалам признаков, по-хорошему можно было сделать вот это: добить StandardScaler(with_std=True), а так код хороший 

## PCA через SVD

In [35]:
def my_pca_svd(X, n_components=None, variance_threshold=0.95):
    """
    PCA через сингулярное разложение (SVD).
    """
    mean = np.mean(X, axis=0) # Объяснение было выше :)
    X_centered = X - mean

    U, s, Vt = np.linalg.svd(X_centered, full_matrices=False) # Сингулярное разложение SVD центрированной матрицы

    n_samples = X.shape[0]
    explained_variance = (s ** 2) / (n_samples - 1) # Расчёт дисперсии
    
    # Автоматический выбор числа компонент
    if n_components is None:
        cumsum_var = np.cumsum(explained_variance) / np.sum(explained_variance)
        n_components = np.argmax(cumsum_var >= variance_threshold) + 1

    components = Vt[:n_components].T  # Главные компоненты 
    transformed = U[:, :n_components] * s[:n_components]  # Проекция данных 

    return transformed, explained_variance[:n_components], components, mean


## Загрузка, сравнение о оценка качества 

(для своего удобства все в одно место столкнул тк вывод результатов и так удобней смотреть)

In [36]:
# Загружаем стандартный набор данных (даже классический)
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Сравнение: sklearn.PCA
scaler_sk = StandardScaler(with_mean=True, with_std=False)  # только центрирование 
X_train_sk = scaler_sk.fit_transform(X_train)
pca_sk = PCA(n_components=0.95)
X_train_pca_sk = pca_sk.fit_transform(X_train_sk)
X_test_pca_sk = pca_sk.transform(scaler_sk.transform(X_test))


X_train_pca_eig, ev_eig, comp_eig, mean_eig = my_pca_eig(X_train, variance_threshold=0.95) # Сравнение: PCA (eig)
X_test_pca_eig = (X_test - mean_eig) @ comp_eig

X_train_pca_svd, ev_svd, comp_svd, mean_svd = my_pca_svd(X_train, variance_threshold=0.95) # Сравнение: PCA (SVD)
X_test_pca_svd = (X_test - mean_svd) @ comp_svd


# Оценка качества через классификацию
def evaluate_pca(X_tr, X_te, y_tr, y_te):
    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1) # простой классификатор, вообще не навороченный, ну прям очень 
    clf.fit(X_tr, y_tr)
    return accuracy_score(y_te, clf.predict(X_te))

acc_sk = evaluate_pca(X_train_pca_sk, X_test_pca_sk, y_train, y_test)
acc_eig = evaluate_pca(X_train_pca_eig, X_test_pca_eig, y_train, y_test)
acc_svd = evaluate_pca(X_train_pca_svd, X_test_pca_svd, y_train, y_test)


#Барабанная дробь --> Вывод результатов 

print("__________СРАВНЕНИЕ_РЕЗУЛЬТАТОВ_PCA__________\n")

print(f".             Число компонент:")
print(f".             sklearn.PCA: {pca_sk.n_components_}")
print(f".             my_pca_eig:  {X_train_pca_eig.shape[1]}")
print(f".             my_pca_svd:  {X_train_pca_svd.shape[1]}\n")


print(f"Объяснённая дисперсия (первые 3 значения):")
print(f"  sklearn: {pca_sk.explained_variance_[:3]}")
print(f"  eig:     {ev_eig[:3]}")
print(f"  svd:     {ev_svd[:3]}\n")

print(f"Макс. разница в объяснённой дисперсии (eig vs sklearn): {np.max(np.abs(ev_eig - pca_sk.explained_variance_)):.2e}")
print(f"Макс. разница в объяснённой дисперсии (svd vs sklearn): {np.max(np.abs(ev_svd - pca_sk.explained_variance_)):.2e}\n")

# Сравнение компонент (с учётом неоднозначности знака)
def align_signs(W):
    # Делаем первый ненулевой элемент каждого вектора положительным
    signs = np.sign(W[np.argmax(np.abs(W), axis=0), range(W.shape[1])])
    return W * signs

comp_sk_aligned = align_signs(pca_sk.components_.T)
comp_eig_aligned = align_signs(comp_eig)
comp_svd_aligned = align_signs(comp_svd)

print(f"Макс. разница в главных компонентах (eig vs sklearn): {np.max(np.abs(comp_eig_aligned - comp_sk_aligned)):.2e}")
print(f"Макс. разница в главных компонентах (svd vs sklearn): {np.max(np.abs(comp_svd_aligned - comp_sk_aligned)):.2e}\n")

print(f"Точность классификации Random Forest:")
print(f"  sklearn.PCA: {acc_sk:.4f}")
print(f"  my_pca_eig:  {acc_eig:.4f}")
print(f"  my_pca_svd:  {acc_svd:.4f}")

__________СРАВНЕНИЕ_РЕЗУЛЬТАТОВ_PCA__________

.             Число компонент:
.             sklearn.PCA: 1
.             my_pca_eig:  1
.             my_pca_svd:  1

Объяснённая дисперсия (первые 3 значения):
  sklearn: [450158.47525092]
  eig:     [450158.47525092]
  svd:     [450158.47525092]

Макс. разница в объяснённой дисперсии (eig vs sklearn): 0.00e+00
Макс. разница в объяснённой дисперсии (svd vs sklearn): 3.49e-10

Макс. разница в главных компонентах (eig vs sklearn): 0.00e+00
Макс. разница в главных компонентах (svd vs sklearn): 7.77e-16

Точность классификации Random Forest:
  sklearn.PCA: 0.8655
  my_pca_eig:  0.8655
  my_pca_svd:  0.8655


# Задание №2
Решите описанную выше задачу методом Ньютона.

Напишите код.

Получите решение.

Сравните с полученными выше.

Расширьте задачу (увеличьте число товаров) и также решите ее тем же кодом.

То есть ваш код должен быть максимально универcальным.


In [37]:
def excess_demand(p, A, b, c): # Функция избыточного спроса (многотоварная)
    return np.exp(-A @ p) + c - b * np.sqrt(p)

# Я только потом увидел, что где-то есть функция Якобиана, но уже все, переписывать...ну не знаю 
def jacobian_e(p, A, b):
    n = len(p) # размерность (число товаров/цен)
    J = np.zeros((n, n))
    for i in range(n): # цикл, а так циклы злооооооооо
        exp_term = np.exp(-A[i] @ p)  # Общий множитель для i-го уравнения
        for j in range(n):
            if i == j:
                J[i, j] = -A[i, j] * exp_term - 0.5 * b[i] / np.sqrt(p[j])
            else:
                J[i, j] = -A[i, j] * exp_term
    return J


def newton_multivariate(F, J, x0, A, b, c, tol=1e-10, max_iter=50): # метод Ньютона 
    x = np.array(x0, dtype=float)
    for i in range(max_iter):
        Fx = F(x, A, b, c)
        norm_F = np.linalg.norm(Fx)
        if norm_F < tol:
            print(f"Сходимость достигнута за {i} итераций --> Всем котятам по рюмке молока за счет заведения! ||F|| = {norm_F:.2e}")
            return x
        try:
            Jx = J(x, A, b)
            delta = np.linalg.solve(Jx, -Fx)
        except np.linalg.LinAlgError:
            raise RuntimeError("Якобиан вырожден на итерации {i}.")
        x = x + delta
    raise RuntimeError(f"Не достигнута сходимость за {max_iter} итераций. Последняя ||F|| = {norm_F:.2e}")


def solve_market_equilibrium(n_goods, seed=42): # Вспомогательная функция для решения задачи любого размера
    np.random.seed(seed)

    A = np.random.rand(n_goods, n_goods) * 0.5 + 0.1  # Все элементы > 0
    b = np.ones(n_goods)
    c = np.ones(n_goods)
    p0 = np.ones(n_goods) 
    
    print(f"\n--- Решение для {n_goods} товаров ---")
    
    p_newton = newton_multivariate(excess_demand, jacobian_e, p0, A, b, c) # Решение своим методом Ньютона
    residual_newton = excess_demand(p_newton, A, b, c)
    
    sol_scipy = root(lambda p: excess_demand(p, A, b, c), p0, jac=lambda p: jacobian_e(p, A, b), method='hybr') # Решение через scipy.optimize.root (для сравнения)
    p_scipy = sol_scipy.x
    residual_scipy = excess_demand(p_scipy, A, b, c)
    

    print(f"Равновесные цены (Ньютон): {p_newton}")
    print(f"Равновесные цены (SciPy):   {p_scipy}")
    print(f"Разница в ценах:           {np.abs(p_newton - p_scipy)}")
    print(f"Макс. невязка (Ньютон):    {np.max(np.abs(residual_newton)):.2e}")
    print(f"Макс. невязка (SciPy):     {np.max(np.abs(residual_scipy)):.2e}")
    
    return p_newton, p_scipy, A, b, c


if __name__ == "__main__":
    # Исходная задача из Семинара 4 (2 товара), все стырено конечно же 
    p_newton_2, p_scipy_2, A2, b2, c2 = solve_market_equilibrium(n_goods=2, seed=42)
    p_newton_5, p_scipy_5, A5, b5, c5 = solve_market_equilibrium(n_goods=5, seed=42) # Расширенная задача (5 товаров)
    p_newton_10, p_scipy_10, A10, b10, c10 = solve_market_equilibrium(n_goods=10, seed=42) # 10 задач



--- Решение для 2 товаров ---
Сходимость достигнута за 4 итераций --> Всем котятам по рюмке молока за счет заведения! ||F|| = 2.72e-13
Равновесные цены (Ньютон): [1.57899742 1.57590756]
Равновесные цены (SciPy):   [1.57899742 1.57590756]
Разница в ценах:           [3.09974268e-13 3.11750625e-13]
Макс. невязка (Ньютон):    1.93e-13
Макс. невязка (SciPy):     0.00e+00

--- Решение для 5 товаров ---
Сходимость достигнута за 4 итераций --> Всем котятам по рюмке молока за счет заведения! ||F|| = 2.50e-15
Равновесные цены (Ньютон): [1.18381597 1.23153726 1.27443827 1.37234634 1.34538808]
Равновесные цены (SciPy):   [1.18381598 1.23153726 1.27443827 1.37234634 1.34538808]
Разница в ценах:           [8.12233392e-10 4.66922057e-11 1.77444059e-09 1.02668052e-09
 1.15661702e-10]
Макс. невязка (Ньютон):    1.33e-15
Макс. невязка (SciPy):     8.72e-10

--- Решение для 10 товаров ---
Сходимость достигнута за 3 итераций --> Всем котятам по рюмке молока за счет заведения! ||F|| = 3.06e-12
Равновесные

# Задание №3

Узнав, сколько кроликов будет у вас через три года, вы потеряли дар речи и бросились строить бизнес планы.

Вы планируете ежемесячно пускать на 4 килограмма диетического мяса и 1 шкурку 100 кроликов. Вы планируете получать выручку в размере 200000 денег.

Найдите цены одного килограмма мяса и одной шкурки если известно, что:

цены являются целыми числами! См. диофантовы уравнения.
спрос на мясо равен 𝑑meat=650−𝑝meat,
спрос на шкурки равен 𝑑fur=100000/𝑝fur.

In [38]:
def solve_rabbit_prices():
    Q_meat = 400  
    Q_fur = 100   
    TARGET_REVENUE = 200000
    print(f"Ежемесячное предложение: {Q_meat} кг мяса, {Q_fur} шкурок")
    print(f"Целевая выручка: {TARGET_REVENUE} денег\n")


    for p_meat in range(1, 650): # тк при p = 650 спрос 0 
        demand_meat = 650 - p_meat
        sold_meat = min(Q_meat, demand_meat)
        revenue_meat = p_meat * sold_meat

        
        if revenue_meat >= TARGET_REVENUE: # своеобразный стопер от бессмысленного перебора 
            continue

        remaining = TARGET_REVENUE - revenue_meat # Расчет оставшейся выручки, которую необходимо получить от шкурок

        for p_fur in range(1, 100001): # при p_fur = 100001 спрос станет 0

            demand_fur = 100000 // p_fur
            if demand_fur == 0:
                break  # Дальнейший рост p_fur не даст спроса, стопер 
            
            sold_fur = min(Q_fur, demand_fur)
            revenue_fur = p_fur * sold_fur

            if revenue_fur == remaining:
                print("Ого, мы нашли решение:")
                print(f"Цена мяса: {p_meat} денег/кг")
                print(f"Цена шкурки: {p_fur} денег/шт")
                print(f"Продано мяса: {sold_meat} кг (спрос = {demand_meat})")
                print(f"Продано шкурок: {sold_fur} шт (спрос = {demand_fur})")
                print(f"Выручка: {revenue_meat + revenue_fur}")
                return p_meat, p_fur

    print("Ого, мы не смогли найти целочисленное решение")
    return None, None

solve_rabbit_prices()

Ежемесячное предложение: 400 кг мяса, 100 шкурок
Целевая выручка: 200000 денег

Ого, мы нашли решение:
Цена мяса: 250 денег/кг
Цена шкурки: 1000 денег/шт
Продано мяса: 400 кг (спрос = 400)
Продано шкурок: 100 шт (спрос = 100)
Выручка: 200000


(250, 1000)

# Материалы и Литература 

- Лекции и Семинары 
- YOUTUBE 
- https://github.com/kshilin/machine-learning/blob/main/11-lesson-unsupervised/ML-11-01-PCA.ipynb 
